In [1]:
import gradio as gr
import openai
import pandas as pd
import os
#import config

from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Acceder a la API key
api_key = os.getenv("API_KEY")
openai.api_key = api_key

c:\Users\plane\OneDrive\Escritorio\COMPUTING SCIENCE\Apuntes_plantillas\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
response = openai.embeddings.create(
    input="Your text string goes here",
    model="text-embedding-ada-002"
)

print(response.data[0].embedding)
print(len(response.data[0].embedding))
print(type(response.data[0].embedding))

[-0.007021795958280563, -0.0052579473704099655, 0.011873218230903149, -0.024888738989830017, -0.024579644203186035, 0.03972522169351578, -0.010065694339573383, -0.00940719060599804, -0.013257419690489769, -0.01002537738531828, -0.011732110753655434, 0.007747493218630552, -0.01416454091668129, 0.007760932203382254, 0.010280715301632881, -0.005053005181252956, 0.022940106689929962, -0.0015496666310355067, 0.015024627558887005, -0.010388226248323917, 0.004844703245908022, 0.01245780847966671, 0.004844703245908022, 0.01089218258857727, -0.006692544091492891, -0.0003231621813029051, 0.005570400506258011, -0.012578757479786873, 0.016381951048970222, 0.0044885738752782345, 0.006605191621929407, -0.00718306191265583, -0.015091821551322937, -0.006581673864275217, -0.018505288287997246, 0.0041694012470543385, 0.003178286598995328, -0.018975647166371346, 0.030291153118014336, -0.007465277798473835, 0.008117061108350754, 0.009521421045064926, -0.0011330625275149941, -0.00042584334732964635, -0.008

In [18]:
#list(openai.embeddings.__dir__())

## Funciones básicas para embeddings en openai

In [2]:
def get_embedding(mi_texto, model= "text-embedding-ada-002"):
    text = mi_texto.replace('\n', ' ')
    respuesta = openai.embeddings.create(input= text, model= model)
    return respuesta.data[0].embedding


def cosine_similarity(embedding1, embedding2): 
    import numpy as np
    from numpy.linalg import norm
    cos_sim = np.dot(embedding1, embedding2)/(norm(embedding1)*norm(embedding2))
    return cos_sim

## algunas pruebitas 

In [4]:
palabras = ['gato', 'casa', 'perro', 'leon', 'tigre', 'aguila', 'colibri', 'cocodrilo']

emb_dict = {}
for palabra in palabras: 
    emb_dict[palabra] = get_embedding(palabra)

In [5]:
emb_dict.keys()

dict_keys(['gato', 'casa', 'perro', 'leon', 'tigre', 'aguila', 'colibri', 'cocodrilo'])

In [6]:
palabra_random= 'kamehameha'
emb_palabra_random = get_embedding(palabra_random)
for palabra in palabras:

    similitud= cosine_similarity(emb_palabra_random, emb_dict[palabra])
    print(f'La similitud entre las palabras {palabra_random} y {palabra} es de {similitud}')

La similitud entre las palabras kamehameha y gato es de 0.7737754486678619
La similitud entre las palabras kamehameha y casa es de 0.7510161630292431
La similitud entre las palabras kamehameha y perro es de 0.7442570677792189
La similitud entre las palabras kamehameha y leon es de 0.773696117295408
La similitud entre las palabras kamehameha y tigre es de 0.7706803292576914
La similitud entre las palabras kamehameha y aguila es de 0.765731918544365
La similitud entre las palabras kamehameha y colibri es de 0.7666075893657508
La similitud entre las palabras kamehameha y cocodrilo es de 0.7734203397306434


In [7]:
goku = get_embedding('goku')
print(cosine_similarity(goku, emb_palabra_random))

0.8380285344373376


## sumas entre embeddings

In [8]:
import numpy as np

In [9]:
print(type(emb_dict['perro']))
print(len(emb_dict['perro']))
print(len(emb_dict['casa']))

<class 'list'>
1536
1536


In [10]:
suma= (np.add(emb_dict['perro'], emb_dict['casa']))
print(len(suma))

1536


In [11]:
for key, value in emb_dict.items():
    print(f'{key}: {cosine_similarity(suma, value)}')

gato: 0.9137955953616776
casa: 0.9608112062589996
perro: 0.960811204496033
leon: 0.8267526388319334
tigre: 0.8745683368240751
aguila: 0.8753674595816591
colibri: 0.8372459169802696
cocodrilo: 0.8623520317720227


In [13]:
pd.DataFrame(emb_dict["leon"])

,0
0,0.023615
1,0.005772
2,0.013493
3,-0.035825
4,-0.021885
...,...
1531,0.037408
1532,-0.009652
1533,-0.015435
1534,0.009066


In [12]:
sumados = (pd.DataFrame(emb_dict["leon"])) + pd.DataFrame(emb_dict['perro'])
print(len(sumados))

for key, value in emb_dict.items(): 
    print(key, ':', cosine_similarity(emb_dict[key], sumados))

1536
gato : [0.89265556]
casa : [0.85973765]
perro : [0.94936958]
leon : [0.94936958]
tigre : [0.88097337]
aguila : [0.88007693]
colibri : [0.84973965]
cocodrilo : [0.8615404]


## Aplicación de un chatbox

In [14]:
def embed_text(path): 
    conocimiento_df = pd.read_csv(path)                                                       # le aplica el path
    conocimiento_df['Embedding']= conocimiento_df['texto'].apply(lambda x: get_embedding(x))  # extrae una nueva columna con los embeddignss
    conocimiento_df.to_csv('embeddings.csv')                                                  # lo pasa a csv
    return conocimiento_df

def buscar(busqueda, datos, n_resulatdos= 5): 
    busqueda_embed = get_embedding(busqueda)                                                      # se extrae el embedding de la pregunta del usuario
    datos['similitud']= datos['Embedding'].apply(lambda x: cosine_similarity(x, busqueda_embed))  # se saca la similitud de la pregunta con las posibles respuestas
    datos = datos.sort_values('similitud', ascending= False)                                      # los ordena de mayor similitud a menor
    return datos.iloc[:n_resulatdos][['texto', 'similitud', 'Embedding']]                         # devuelve estas columnas y los primeros n resultados, este formato de iloc es obligatorio

In [17]:
texto_emb = embed_text("data/chatbot_qa.csv")

display(texto_emb)

,texto,Embedding
0,Cerramos los domingos,"[0.005177388433367014, -0.0031512435525655746,..."
1,Estamos abiertos de Lunes a Sabado de 9 a 16:00,"[-0.01700492762029171, -0.0039050509221851826,..."
2,Las citas se pueden agendar al: 523 432 4212,"[-0.017580052837729454, -0.007565770763903856,..."
3,Contamos con servicios de carpinteria y herrería,"[-0.0018300488591194153, 0.009386846795678139,..."
4,No contamos con servicio de jardinería,"[-0.00400054594501853, -0.0003356921370141208,..."
5,Las cotizaciones se hacen una vez que ya se de...,"[0.0038105526473373175, -2.1932393792667426e-0..."


In [ ]:
with gr.Blocks() as demo: 
    busqueda = gr.Textbox(label= 'Buscar')
    output = gr.DataFrame(headers= ['texto'])
    greet_btn = gr.Button('preguntar')
    greet_btn.click(fn= buscar, inputs=[busqueda, gr.DataFrame(texto_emb)], outputs=output)

demo.launch(share= False)

## Procesar datos de un PDF

In [63]:
# pip install langchain pypdf
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

loader = PyPDFLoader('./LogicaBorrosa.pdf')
pages = loader.load_and_split()

In [64]:
# Mostramos todo el contenido por páginas
for page in pages: 
    print(page.page_content)
    # print('\n')

La lógica borrosa  
y sus 
aplicaciones  
 
 
 
 
 
 
 
 
 
La Teorí a de C onjunt os 
Borrosos fue i ntroduci da por Lotfi A. Zadeh (Azerbai yán, 1921, 
actualmente profesor e
mérito de la Universidad de 
California en Berkeley) a m ediados de 
los años 60. Previ amente, Max Black 
(1909 - 1989), en un art ículo de 1937 
titulado "Vag ueness: An  exercise in  
Logi cal Anal ysis" y Karl  Menger (1902 
- 1985) con l os artículos de 1942 
"Statistical Metrics" y lo s de los años 
50 sobre rel aciones borrosas de 
indistinguibilidad, se ntaron las bases de 
lo que hoy es una teoría tan  utilizad a y 
con t an buenos resul tados. 
Bajo el concept o de C onjunt o Borroso 
(Fuzzy Set ) resi de la idea de que l os 
elem entos clave en el pensam iento 
humano no son núm eros, si no et iquetas 
lingüísticas. Est as etiquetas perm iten 
que los objetos pasen de pertenecer de 
una clase a otra de form a suave y 
flexible. 
La Lógi ca Borrosa se puede i nscribir 
en el contexto de la Lógica 
Mult

In [65]:
# para poder dividir estas páginas en parrafos o bloques

split = CharacterTextSplitter(  # CREAMOS la función para que nos parta el texto, pero todavía no lo parte
    chunk_size= 600,            # el numero de caracteres que nos corte en cada uno de los bloques, dependiendo del caso de uso debe aumentar o disminuir
    separator='.')            # el separador que queremeos que encuentre en este caso Punto. para  PUNTO Y APARTE ---># '.\n'

textos = split.split_documents(pages)  # le pasamos las paginas que queremos que parta

In [66]:
print(textos)

[Document(page_content='La lógica borrosa  \ny sus \naplicaciones  \n \n \n \n \n \n \n \n \n \nLa Teorí a de C onjunt os \nBorrosos fue i ntroduci da por Lotfi A. Zadeh (Azerbai yán, 1921, \nactualmente profesor e\nmérito de la Universidad de \nCalifornia en Berkeley) a m ediados de \nlos años 60', metadata={'source': './LogicaBorrosa.pdf', 'page': 0}), Document(page_content='Zadeh (Azerbai yán, 1921, \nactualmente profesor e\nmérito de la Universidad de \nCalifornia en Berkeley) a m ediados de \nlos años 60. Previ amente, Max Black \n(1909 - 1989), en un art ículo de 1937 \ntitulado "Vag ueness: An  exercise in  \nLogi cal Anal ysis" y Karl  Menger (1902 \n- 1985) con l os artículos de 1942 \n"Statistical Metrics" y lo s de los años \n50 sobre rel aciones borrosas de \nindistinguibilidad, se ntaron las bases de \nlo que hoy es una teoría tan  utilizad a y \ncon t an buenos resul tados', metadata={'source': './LogicaBorrosa.pdf', 'page': 0}), Document(page_content='Bajo el concept o d

In [67]:
print(textos[0].page_content)

La lógica borrosa  
y sus 
aplicaciones  
 
 
 
 
 
 
 
 
 
La Teorí a de C onjunt os 
Borrosos fue i ntroduci da por Lotfi A. Zadeh (Azerbai yán, 1921, 
actualmente profesor e
mérito de la Universidad de 
California en Berkeley) a m ediados de 
los años 60


In [68]:
textos = [str(i.page_content) for i in textos]          # extraeos los textos y los pasamos a listas
parrafos = pd.DataFrame(textos, columns= ['texto'])    # lo pasamos a dataframe
print(parrafos)

                                                texto
0   La lógica borrosa  \ny sus \naplicaciones  \n ...
1   Zadeh (Azerbai yán, 1921, \nactualmente profes...
2   Bajo el concept o de C onjunt o Borroso \n(Fuz...
3   Adem ás, \nadelantaba una lógica de valores ci...
4   ¿Qué son los conjuntos \nborrosos?. \nEn un co...
..                                                ...
60  • Técnicas avanzadas para describir \n“perfi l...
61  • Com ercio electrónico basado en \ntécnicas d...
62  • Mejoras en l os m étodos de \nagregación de ...
63  • Agent es inteligentes aut ónom os para \nrea...
64  Y un l argo et cétera de nuevos cam pos \nde a...

[65 rows x 1 columns]


In [72]:
# vamos a extraer el embedding de cada parrafo

parrafos['embedding'] = parrafos['texto'].apply(lambda x: get_embedding(x))
parrafos.to_csv('Logica_borrosa.csv')

In [73]:
mi_pdf_emb = embed_text("./Logica_borrosa.csv")
with gr.Blocks() as demo: 
    busqueda = gr.Textbox(label= 'Buscar')
    output = gr.DataFrame(headers= ['texto'])
    greet_btn = gr.Button('preguntar')
    greet_btn.click(fn= buscar, inputs=[busqueda, gr.DataFrame(mi_pdf_emb)], outputs=output)

demo.launch(share= False)

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


## últimos consejos

### 1

Un consejo qué les doy a todos para almacenar los archivos con los embediings es que nos los guarden en csv, sino en un archivo pickle, de esta manera al leer el archivo nos aseguramos de que los embediings se lean como arreglos de numpy y no como un str.
Para hacer eso con el mismo pandas: pd.to_pickle('nameFile.pkl')
Para leerlo se usa: pd.read_pickle('nameFile.pkl')